## Fichero para generar las imagenes stitcheadas de todas las partes de cada corte de la medula

In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import itertools
import os

Primero de todo generaremos las imagenes procesadas que utilizaremos para el stitching de cada una de las partes de cada corte.

In [2]:
directorio_actual = os.getcwd()

ruta_parts = os.path.join(directorio_actual, "Parts")

if os.path.exists(ruta_parts):
    directorios = os.listdir(ruta_parts)

    for directorio in directorios:
        ruta_directorio = os.path.join(ruta_parts, directorio)

        if os.path.isdir(ruta_directorio):
            ruta_bda = os.path.join(ruta_directorio, "bda")

            if os.path.exists(ruta_bda):
                imgs = [archivo for archivo in os.listdir(ruta_bda) if archivo.endswith(".tif")]

                carpeta_guardado_cv2 = os.path.join(ruta_bda, "imgs_processed_v3")
                os.makedirs(carpeta_guardado_cv2, exist_ok=True)

                os.chdir(ruta_bda)

                imagenes_procesadas = []

                for i, img in enumerate(imgs):
                    ruta_imagen = os.path.join(ruta_bda, img)
                    imagen = cv2.imread(ruta_imagen)

                    canal_rojo = imagen[:, :, 2]

                    nombre_archivo_cv2 = os.path.join(carpeta_guardado_cv2, f'processed_p{i+1}.jpg')
                    cv2.imwrite(nombre_archivo_cv2, canal_rojo)

                    imagenes_procesadas.append(canal_rojo)

                os.chdir(directorio_actual)


Generamos una carpeta de outputs con los diferentes stitchings de cada imagen.
En caso de error modificar el rango de combinaciones, ya que hay algunas combinaciones donde el kernel da error directamente.

In [3]:
carpeta_principal = "Parts"

stitcher = cv2.Stitcher.create(mode=0)

for directorio in os.listdir(carpeta_principal):
    ruta_directorio = os.path.join(carpeta_principal, directorio)

    if os.path.isdir(ruta_directorio):
        ruta_imgs_procesadas = os.path.join(ruta_directorio, "bda", "imgs_processed_v3")

        imgs_p = [archivo for archivo in os.listdir(ruta_imgs_procesadas) if archivo.endswith(".jpg")]

        combinaciones = list(itertools.permutations(imgs_p, 6))

        carpeta_resultados = os.path.join(ruta_directorio, "bda", "stitching_processed_v3")
        os.makedirs(carpeta_resultados, exist_ok=True)

        num_stitchings_exitosos = 0

        for i, combinacion in enumerate(combinaciones[10:30]):
            imagenes = [cv2.imread(os.path.join(ruta_imgs_procesadas, ruta)) for ruta in combinacion]

            resultado, imagen_stitched = stitcher.stitch(imagenes)

            if resultado == cv2.Stitcher_OK:
                num_stitchings_exitosos += 1

                nombre_salida = f"stitched_{num_stitchings_exitosos}.jpg"
                ruta_salida = os.path.join(carpeta_resultados, nombre_salida)

                cv2.imwrite(ruta_salida, imagen_stitched)
